Fitting critical current graphs

Author: Cliff Sun

In [91]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import pandas as pd

In [92]:
# TODO: Convert data in pandas dataframe

# TODO: Make while loop that continuously feeds in random arrays with random dimensions with
# a random number of junctions

# TODO: Calculate cost by adding the squared differences

# TODO: Any cost under a specified threshold is added to a folder called "Candidate Set-ups"

# TODO: Run until a certain amount of graphs are in the folder, maybe 20 or so

In [93]:
def current(B, arrJ, arrC, y): # y is initial phase difference of the whole circuit, B is the magnetic field, arrJ is the location of junctions, arrC is critical current associated with each junction
    curr = 0 # summation of all currents in the entire junction
    limit = int(len(arrJ) / 2) # number of junctions in the SQUID
    numOfSegments = 0
    for n in range(limit):
        numOfSegments = 5 if (arrJ[2 * n + 1] - arrJ[2 * n]) < 0.05 else int(100 * (arrJ[2 * n + 1] - arrJ[2 * n]))
        sizeOfSegment = float((arrJ[2 * n + 1] - arrJ[2 * n]) / numOfSegments)
        for i in range(numOfSegments):
            curr += arrC[n] * np.sin(y + (2 * np.pi * B) * (arrJ[2 * n] + i * sizeOfSegment)) * (1/numOfSegments)

    # phase difference evolves according to 2 * pi * B

    # curr += (critical current element in array)(sin(y + (2 * pi * B) * length)
    return curr

In [94]:
def maxCurrent(B, arrayJ, arrayC): # Spits out the maximum current by varying the gauge invariant phase of the left end (free parameter) gamma
    Y=np.linspace(0, 2*np.pi, 150)
    dummyArray=[]
    for gamma in Y:
        dummyArray.append(current(B, arrayJ, arrayC, gamma))
    return max(dummyArray)

In [95]:
# MagField = np.linspace(-10, 10, 10000) # an array of Magnetic Fields ranging from 0 to 100 with 5000 total elements

The 2 lines below is where the useful section of the code is for modeling a SQUID

In [96]:
IMaxPoint = []

In [97]:
# length = len(MagField)
# for i in range(length):
#     IMaxPoint.append(maxCurrent(MagField[i], arrayOfJunctions, criticalCurrents) / np.sum(criticalCurrents)) # This integer represents the number of segments you want to cut each junction up into (the higher the number, the better the approximation)

In [98]:
def count_images_in_folder(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg']  # Add more extensions if needed
    image_count = 0

    # List all files in the folder
    files = os.listdir(folder_path)

    # Iterate through each file and check if it's an image
    for file in files:
        if os.path.isfile(os.path.join(folder_path, file)):
            if any(file.lower().endswith(ext) for ext in image_extensions):
                image_count += 1

    return image_count

def cost_function(basis_data, generated_data):
    diff = basis_data - generated_data
    for i in range(len(diff)):
        diff[i] = diff[i]**2
    return np.sum(diff)

def generate_random_junction_number(low_bound, top_bound):
    return int(random.uniform(low_bound, top_bound))

def generate_random_junction_coordinates(numOfJunctions):
    random_numbers = [random.uniform(0, 1) for _ in range(numOfJunctions)]
    return random_numbers

def criticalCurrent(density, arrJ):
    criticalCurrents = []
    junctionWidths = []
    for i in range(len(arrJ)//2):
        junctionWidths.append(arrJ[2*i+1] - arrJ[2*i])
    for i in range (len(junctionWidths)):
        criticalCurrents.append(junctionWidths[i] * density[i])
    return criticalCurrents, junctionWidths


def array_to_python_syntax(arr):
    if not arr.any():
        return "[]"

    result = "["
    for item in arr:
        if isinstance(item, str):
            result += f"'{item}', "
        else:
            result += f"{item}, "
    
    # Remove the trailing comma and space
    result = result[:-2]
    result += "]"
    return result

def write_txt_file(directory, arrJ_gen, cost, file_name):
    converted_arrJ_gen = array_to_python_syntax(arrJ_gen)
    text_content = f"Cost = {cost}. \n \n The generated Junction array is {converted_arrJ_gen}."
    numOfJunctions = len(arrJ_gen)//2
    text_file_path = os.path.join(directory, file_name)
    with open(text_file_path, 'w') as file:
        file.write(text_content)

def create_folder(repository_path, folder_name):
    folder_path = os.path.join(repository_path, folder_name)
    
    # Check if the folder already exists
    if not os.path.exists(folder_path):
        # Create the folder
        os.makedirs(folder_path)
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Folder '{folder_name}' already exists.")
    return folder_path

def junction_width(arrJ):
    junctionWidths = []
    for i in range(len(arrJ)//2):
        junctionWidths.append(arrJ[2*i+1] - arrJ[2*i])
    return junctionWidths

def save_plot(x_values, y_values, directory, file_name):
    experimental_df = pd.read_csv(r"experimental data\experimental csvs\5.0uV_normalized.csv")
    experimental_x = np.array(experimental_df['B'])
    experimental_y = np.array(experimental_df['I_c'])

    # Plot the data
    plt.plot(x_values, y_values)
    plt.plot(experimental_x, experimental_y)
    plt.xlabel('B')
    plt.ylabel('I_c')

    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    # Save the plot
    plot_path = os.path.join(directory, file_name)
    plt.savefig(plot_path)

    # Close the plot to release resources
    plt.close()

In [99]:
folder_path = "experimental data\candidate set-ups"
model_data_csv = pd.read_csv(r"experimental data\experimental csvs\5.0uV_normalized.csv")
model_i_c = np.array(model_data_csv['I_c'])
model_B = np.array(model_data_csv['B'])

top_bound = 2
bottom_bound =5

In [100]:
while(True):
    # numOfJunctions = generate_random_junction_number(low_bound=bottom_bound, top_bound=top_bound)
    numOfJunctions = 4
    arrayOfJunctions = generate_random_junction_coordinates(numOfJunctions= (2 * numOfJunctions))
    arrayOfJunctions.sort()
    arrayOfJunctions[0] = 0
    arrayOfJunctions[-1] = 1
    criticalCurrents, junctionWidths = criticalCurrent(np.ones(numOfJunctions),arrayOfJunctions)
    sum = np.sum(criticalCurrents)

    for i in range(len(model_B)):
        IMaxPoint.append(maxCurrent(model_B[i], arrayOfJunctions, criticalCurrents) / sum)
    # plt.plot(model_B, IMaxPoint)
    # plt.plot(model_B, model_i_c)
    # plt.show()
    cost = cost_function(IMaxPoint, model_i_c)
    print(cost)
    print(numOfJunctions)
    print(arrayOfJunctions)

    if (cost < 30):
        folder_name = f"{cost}_{numOfJunctions}"
        file_name = f"{{{cost}_{numOfJunctions}}}"
        repo_path = r"experimental data\candidate set-ups\regular cost"
        directory = os.path.join(repo_path, folder_name)
        folder_path = create_folder(repository_path=repo_path, folder_name=folder_name)
        write_txt_file(directory=folder_path, arrJ_gen=np.array(arrayOfJunctions), cost=cost, file_name=f"{file_name}.txt")
        save_plot(model_B, IMaxPoint, directory=folder_path, file_name=f"{file_name}.png")
        print("candidate plot found")
    IMaxPoint = []

54.741797172141105
4
[0, 0.43863788429892836, 0.4603135807016875, 0.622278423106717, 0.6874645080480014, 0.8601386352435921, 0.8877803471557899, 1]
154.86392084860768
4
[0, 0.2411355020304361, 0.3983496078861398, 0.4395644125269601, 0.564737897186116, 0.5670820335319658, 0.6955812492703659, 1]
164.66641122242777
4
[0, 0.2564974281533068, 0.5330740866744664, 0.5738558887859447, 0.7111613612386034, 0.7354078297856373, 0.8578978942735556, 1]
79.36800134348648
4
[0, 0.06756501446120822, 0.23155728041701784, 0.27125222799291593, 0.5902752134056956, 0.7378590522540391, 0.7618974356626405, 1]
85.6399912393427
4
[0, 0.19745880943277538, 0.19847509777826688, 0.3225166499566088, 0.5134896657201525, 0.6743795959581198, 0.6768336237206355, 1]
226.94508120654808
4
[0, 0.07770460159484782, 0.13724356836429485, 0.18269746771837192, 0.22807760414378397, 0.322453362359998, 0.8116782975915673, 1]
78.55342764444097
4
[0, 0.28064420636856136, 0.3186621235566539, 0.406809869569873, 0.5078802147271875, 0.56

KeyboardInterrupt: 

In [ ]:
if (cost < 150):
        folder_name = f"{np.std(junction_width(arrJ=arrayOfJunctions))}_{numOfJunctions}"
        file_name = f"{{{cost}_{numOfJunctions}}}"
        repo_path = r"experimental data\candidate set-ups"
        directory = os.path.join(repo_path, folder_name)
        folder_path = create_folder(repository_path=repo_path, folder_name=folder_name)
        write_txt_file(directory=folder_path, arrJ_gen=np.array(arrayOfJunctions), cost=cost, file_name=f"{file_name}.txt")
        save_plot(MagField, IMaxPoint, directory=folder_path, file_name=f"{file_name}.png")
        print("candidate plot found")

Folder '0.006046016466630593_85' already exists.
candidate plot found
